In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.svm import LinearSVC

In [3]:
def dec2bin(i):
    X = [0]*4;
    X[0] = i // 8;
    i = i-(8*X[0])
    X[1] = i // 4;
    i = i-(4*X[1])
    X[2] = i // 2;
    i = i-(2*X[2]);
    X[3] = i;
    return X

In [4]:
def bin2dec(X):
    return 8*X[0]+4*X[1]+2*X[2]+X[3]

In [5]:
def swap(X, Y):
    X[0], Y[0]  = Y[0], X[0]
    X[1], Y[1]  = Y[1], X[1]
    X[2], Y[2]  = Y[2], X[2]
    X[3], Y[3]  = Y[3], X[3]

In [6]:
train_data = np.loadtxt( "train.dat" )
train_data.shape


test_data = np.loadtxt( "test.dat" )
test_data.shape

(40000, 73)

In [7]:
def preprocess(X):
    for row in X:
        if(int(bin2dec(row[64:68])) >= int(bin2dec(row[68:72]))):
            swap(row[64:68], row[68:72])
            row[72] = 1 - row[72]
    return X

train_data = preprocess(train_data)
data = train_data

test_data = preprocess(test_data)

In [10]:
models = [[None]*16]*16

counter = 0

for i in range(16):
    X = dec2bin(i);
    for j in range(i+1,16):
        Y = dec2bin(j)
        mask1 = np.where((data[:,64]==X[0]) & (data[:,65]==X[1]) & (data[:,66]==X[2]) & (data[:,67]==X[3]) & (data[:,68]==Y[0]) & (data[:,69]==Y[1]) & (data[:,70]==Y[2]) & (data[:,71]==Y[3]))
        a = data[mask1[0]]
        
        data_trn, data_tst = train_test_split(a, train_size = 450)

        # Insufficient training data can lead to low accuracy
        # Try using train_size = 30000 or 3000 or 300 or 30

        clf = LinearSVC( loss = "hinge" )
        clf.fit(data_trn[:,:-1], data_trn[:,-1] )
        counter += (np.average( data_tst[:,-1] == clf.predict(data_tst[:,:-1] ) ) )
        
        models[i][j] = clf

print(counter/120)

C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the

C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the

C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the

0.9397817542162508


C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [9]:
a = 0

for row in test_data:
    x = int(bin2dec(row[64:68]))
    y = int(bin2dec(row[68:72]))
    clf = models[x][y]
    a += (row[-1] == clf.predict(row[:-1].reshape(1,-1) ) ) 
        
print(a/40000)

[0.735325]
